In [ ]:
import fa

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F 
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

In [ ]:
class MLP(nn.Module):
    def __init__(self, sizes, func=nn.Linear):
        super().__init__()
        self.layers = nn.ModuleList()
        for n1, n2 in zip(sizes[:-1], sizes[1:]):
            self.layers.append(func(n1, n2))
    
    def forward(self, x):
        for l in self.layers[:-1]:
            x = F.relu(l(x))
        return self.layers[-1](x)
    
net = MLP([1, 5, 5, 1])

In [ ]:
def toT(x):
    return torch.tensor(x, dtype=torch.float32, requires_grad=False)

def fromT(x):
    return x.detach().numpy()

In [ ]:
n = 1000
x = np.linspace(-2, 2, n).reshape(-1, 1)
y = x + 0.1 * np.random.randn(*x.shape)
plt.plot(x, y, 'ro')

In [ ]:
dataset = TensorDataset(toT(x), toT(y))
loader = DataLoader(dataset, batch_size=64, shuffle=True)

In [ ]:
def train(net, loader, epochs=50):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(net.parameters())
    losses = []
    every = 1000
    for epoch in range(epochs):
        running_loss = 0.0
        i = 0

        for batch in loader:
            inputs, targets = batch

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            i+=1
            if i % every == 1:    # print every 2000 mini-batches
                running_loss /= every
                losses.append(running_loss)
                #print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 50))
                running_loss = 0.0

    return losses

In [ ]:
net = MLP([1, 5, 5, 1])
train(net, loader, 100)

In [ ]:
plt.figure(0)

net1 = MLP([1, 5, 5, 1])
loss1 = train(net1, loader, 100)

net2 = MLP([1, 5, 5, 1], func=fa.FALinear)
loss2 = train(net2, loader, 500)
plt.plot(loss1)
plt.plot(loss2)

plt.figure(1)
x = np.linspace(-2, 2, n).reshape(-1, 1)
y2 = fromT(net1(toT(x)))
y3 = fromT(net2(toT(x)))
plt.plot(x, y2)
plt.plot(x, y3)